In [1]:
!pip install /pstore/home/shetabs1/code/iflai/

Defaulting to user installation because normal site-packages is not writeable
Processing /pstore/home/shetabs1/code/iflai
  Created wheel for iflai: filename=iflai-0.0.1-py3-none-any.whl size=14094 sha256=b667ff5856acb62e6aca803c576121e0d6086d5d7f39c87b799879698d74214a
  Stored in directory: /local/2128146/pip-ephem-wheel-cache-tvet8x3e/wheels/77/c9/46/be8aac478e8f67c427d0996a39b13c1213341ddb146b974964
Successfully built iflai
  Attempting uninstall: iflai
    Found existing installation: iflai 0.0.1
    Uninstalling iflai-0.0.1:
      Successfully uninstalled iflai-0.0.1
You should consider upgrading via the '/pstore/apps/Python/3.8.3-2020.07/bin/python -m pip install --upgrade pip' command.


In [2]:
import iflai
import matplotlib.pyplot as plt
import iflai.ml.features as features
from iflai.ml.feature_selection import AutoFeatureSelection
from iflai.ml.feature_extractor import AmnisData
import numpy as np
import pandas as pd
import glob
import warnings
warnings.filterwarnings("ignore")
from tqdm import tqdm 
from sklearn.model_selection import train_test_split
import h5py
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction import DictVectorizer
from iflai.utils import list_of_dict_to_dict
from sklearn.preprocessing import MinMaxScaler 
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
import seaborn as sns

In [3]:
from sklearn.metrics import (plot_confusion_matrix, 
                             matthews_corrcoef, 
                             classification_report,
                             confusion_matrix, 
                             accuracy_score, 
                             balanced_accuracy_score, 
                             cohen_kappa_score, 
                             f1_score,  
                             precision_score, recall_score)
from statsmodels.stats.contingency_tables import mcnemar

def classification_complete_report(estimator, X, y_true, plot = True ): 
    y_pred = estimator.predict(X)
    print(classification_report(y_true, y_pred))
    print(15*"----")
    print("matthews correlation coeff: %.4f" % (matthews_corrcoef(y_true, y_pred)) )
    print("Cohen Kappa score: %.4f" % (cohen_kappa_score(y_true, y_pred)) )
    print("Accuracy: %.4f & balanced Accuracy: %.4f" % (accuracy_score(y_true, y_pred), balanced_accuracy_score(y_true, y_pred)) )
    print("macro F1 score: %.4f & micro F1 score: %.4f" % (f1_score(y_true, y_pred, average = "macro"), f1_score(y_true, y_pred, average = "micro")) )
    print("macro Precision score: %.4f & micro Precision score: %.4f" % (precision_score(y_true, y_pred, average = "macro"), precision_score(y_true, y_pred, average = "micro")) )
    print("macro Recall score: %.4f & micro Recall score: %.4f" % (recall_score(y_true, y_pred, average = "macro"), recall_score(y_true, y_pred, average = "micro")) )
    print(15*"----")
    
    if plot:
        plot_confusion_matrix(estimator, X, y_true,cmap = "Greys", xticks_rotation='vertical')
    

In [4]:
data_path = "/pstore/data/DS4/synapse_formation_full_data/"

feature_extractor = FeatureUnion([
                                ("MaskBasedFeatures", features.MaskBasedFeatures()), 
                                ("GLCMFeatures", features.GLCMFeatures()),  
                                ("GradientRMS", features.GradientRMS()),  
                                ("BackgroundMean", features.BackgroundMean()),  
                                ("CellShape", features.CellShape()),  
                                ("Collocalization", features.Collocalization()),    
                                ("IntersectionProperties", features.IntersectionProperties(channels = [4,5])),   
                                ("CenterOfCellsDistances", features.CenterOfCellsDistances()), 
                                     ])

In [5]:
amnis_data = AmnisData(data_path, feature_extractor,sample_size = 1)

100%|██████████| 1/1 [00:00<00:00, 52.91it/s]

Metadata prepration starts...
Experiment_1 Donor_0 Not_Stimulated_Bcells


Experiment_1 Donor_1 +SEA


100%|██████████| 1/1 [00:00<00:00, 933.73it/s]


Experiment_1 Donor_1 -SEA


...

Experiment_4 Donor_10 Not_Stimulated_Tcells
Experiment_4 Donor_9 +SEA


100%|██████████| 1/1 [00:00<00:00, 987.59it/s]


Experiment_4 Donor_9 -SEA


100%|██████████| 1/1 [00:00<00:00, 1123.57it/s]
0it [00:00, ?it/s]

Experiment_4 Donor_9 Not_Stimulated_Tcell
...metadata prepration ended.


In [6]:
#amnis_data.metadata.to_csv("metadata.csv", index = False)

In [8]:
amnis_data.metadata = pd.read_csv("metadata.csv")

indx = amnis_data.metadata["experiment"] != "Experiment_4"
indx = indx & (amnis_data.metadata["condition"].isin(["-SEA","+SEA"]))
amnis_data.metadata = amnis_data.metadata.loc[indx,:].copy()

In [9]:
all_combinations = amnis_data.metadata.groupby(["experiment", "donor", "condition"]).size().reset_index().iloc[:,:-1]
all_combinations

,experiment,donor,condition
0,Experiment_1,Donor_1,+SEA
1,Experiment_1,Donor_1,-SEA
2,Experiment_1,Donor_2,+SEA
3,Experiment_1,Donor_2,-SEA
4,Experiment_2,Donor_1,+SEA
5,Experiment_2,Donor_1,-SEA
6,Experiment_2,Donor_3,+SEA
7,Experiment_2,Donor_3,-SEA
8,Experiment_2,Donor_4,+SEA
9,Experiment_2,Donor_4,-SEA


In [10]:
k = 10000

indx = amnis_data.metadata["set"] == "labeled"
for i in tqdm(range(len(all_combinations)), position=0, leave=True):
    indx_comb = (amnis_data.metadata["experiment"] == all_combinations.loc[i,"experiment"])
    indx_comb = indx_comb & (amnis_data.metadata["donor"] == all_combinations.loc[i,"donor"])
    indx_comb = indx_comb & (amnis_data.metadata["condition"] == all_combinations.loc[i,"condition"])
    indx_comb = indx_comb[indx_comb].sample(k).index.tolist()
    indx[indx_comb] = True

amnis_data.metadata = amnis_data.metadata.loc[indx,:].reset_index(drop = True).copy()

100%|██████████| 17/17 [00:02<00:00,  6.40it/s]


In [ ]:
amnis_data.extract_features_for_all_samples()

  1%|          | 960/176150 [01:46<5:39:11,  8.61it/s]

In [12]:
## start here

In [13]:
amnis_data.df_features = amnis_data.df_features.fillna(0.)
amnis_data.df_features = amnis_data.df_features.loc[:, amnis_data.df_features.std() > 0.]

In [18]:
column_dict_rename = dict()

channel_names = {   
         "Ch1": "BF",
         "Ch2": "Antibody",
         "Ch3": "CD18",
         "Ch4": "F-Actin",
         "Ch5": "MHCII",
         "Ch6": "CD3",
         "Ch7": "P-CD3zeta",
         "Ch8": "Live-Dead"}

for col in amnis_data.df_features.columns:
    new_col = col
    for ch in channel_names:
        new_col = new_col.replace(ch,channel_names[ch] )
    column_dict_rename[col] = new_col
    
amnis_data.df_features = amnis_data.df_features.rename(columns=column_dict_rename).copy()

In [19]:
amnis_data.metadata.to_csv("metadata_subset.csv", index = False)
amnis_data.df_features.to_csv("df_features.csv", index = False)

In [20]:
amnis_data.df_features.memory_usage(index=True).sum()/1e9

1.970061728